In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
training= 'gdrive/MyDrive/CS4220/Datasets/training-decoy-75.csv.gz'
df_train = pd.read_csv(training)
df_train

,1517,1934,326,2069,730,63,2001,1166,311,1504,...,1958,359,164,1853,910,882,1447,1544,1089,genome_label
0,0.008573,0.000504,0.000000,0.008069,0.000000,0.000504,0.001513,0.000000,0.000000,0.000504,...,0.000504,0.000000,0.000000,0.001513,0.000000,0.000504,0.000504,0.001009,0.001009,0
1,0.004038,0.001346,0.000000,0.002692,0.000000,0.000000,0.002692,0.000000,0.000000,0.000000,...,0.001346,0.000000,0.000000,0.000000,0.000000,0.000000,0.001346,0.002692,0.000000,0
2,0.007321,0.000271,0.000271,0.014100,0.000000,0.000271,0.000542,0.000000,0.000000,0.001627,...,0.000271,0.000000,0.000271,0.002440,0.000000,0.000271,0.000000,0.000542,0.000000,0
3,0.009646,0.000000,0.000000,0.008574,0.000000,0.001072,0.000000,0.000000,0.000000,0.000000,...,0.001072,0.000000,0.001072,0.001072,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.007216,0.000601,0.000000,0.013229,0.000301,0.000601,0.000601,0.000000,0.000000,0.001203,...,0.000000,0.000000,0.000000,0.001203,0.000000,0.000601,0.000301,0.003307,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87171,0.000000,0.000000,0.000000,0.000000,0.006323,0.000903,0.000000,0.000000,0.000000,0.000000,...,0.000903,0.002710,0.000903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
87172,0.000000,0.000424,0.001695,0.000000,0.000424,0.000000,0.000000,0.000847,0.001483,0.000212,...,0.000424,0.000635,0.000424,0.000847,0.000847,0.000000,0.000000,0.000424,0.000000,3
87173,0.000000,0.002593,0.000000,0.000000,0.001235,0.000123,0.000247,0.000741,0.000123,0.000123,...,0.000370,0.000123,0.000741,0.001729,0.000123,0.000000,0.000247,0.000494,0.000123,3
87174,0.000000,0.000000,0.002620,0.000000,0.000000,0.000000,0.001747,0.000873,0.000000,0.000873,...,0.000000,0.000000,0.001747,0.000873,0.000000,0.000000,0.000000,0.000873,0.000000,3


In [3]:
from sklearn import preprocessing
df_y = pd.read_csv('gdrive/MyDrive/CS4220/Datasets/labels.csv', index_col=0)
le = preprocessing.LabelEncoder()
le.fit(df_y['genome_name'].unique())
le.classes_

array(['burkholderia_pseudomallei', 'corynebacterium_diphtheriae',
       'corynebacterium_ulcerans', 'decoy', 'mycobacterium_tuberculosis',
       'mycobacterium_ulcerans', 'neisseria_gonorrhoeae',
       'pseudomonas_aeruginosa', 'staphylococcus_aureus',
       'staphylococcus_pyogenes', 'streptococcus_pneumoniae'],
      dtype=object)

In [4]:
from sklearn.model_selection import train_test_split
import lightgbm
import matplotlib.pylab as plt

X, y = df_train.iloc[:, :-1], df_train.iloc[:, -1]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((69740, 520), (17436, 520), (69740,), (17436,))

In [5]:
train_data = lightgbm.Dataset(X_train, label=y_train)
valid_data = lightgbm.Dataset(X_valid, label=y_valid)   

In [6]:
parameters = {'objective': 'multiclass',
              'num_class': 11,
              'metric': 'multi_logloss',
              'is_unbalance': 'true',
              'boosting': 'gbdt',
              'num_leaves': 31,
              'feature_fraction': 0.5,
              'bagging_fraction': 0.5,
              'bagging_freq': 20,
              'learning_rate': 0.05,
              'n_estimators': 500,
              'max_depth':5,
              'lambda_l1': 1.0,
              'lambda_l2':0.25,
              'verbose': 1
             }

In [7]:
model_lgbm = lightgbm.train(parameters,
                            train_data,
                            valid_sets=valid_data,
                            early_stopping_rounds=50)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 1.01766
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 0.960648
[3]	valid_0's multi_logloss: 0.915145
[4]	valid_0's multi_logloss: 0.876573
[5]	valid_0's multi_logloss: 0.844326
[6]	valid_0's multi_logloss: 0.815474
[7]	valid_0's multi_logloss: 0.789684
[8]	valid_0's multi_logloss: 0.766453
[9]	valid_0's multi_logloss: 0.74525
[10]	valid_0's multi_logloss: 0.725639
[11]	valid_0's multi_logloss: 0.707599
[12]	valid_0's multi_logloss: 0.691184
[13]	valid_0's multi_logloss: 0.67567
[14]	valid_0's multi_logloss: 0.661143
[15]	valid_0's multi_logloss: 0.647214
[16]	valid_0's multi_logloss: 0.634263
[17]	valid_0's multi_logloss: 0.621863
[18]	valid_0's multi_logloss: 0.60997
[19]	valid_0's multi_logloss: 0.598911
[20]	valid_0's multi_logloss: 0.588476
[21]	valid_0's multi_logloss: 0.57788
[22]	valid_0's multi_logloss: 0.567726
[23]	valid_0's multi_logloss: 0.558432
[24]	valid_0's multi_logloss: 0.549229
[25]	valid_0's m

In [8]:
y_train_pred = model_lgbm.predict(X_train)
y_valid_pred = model_lgbm.predict(X_valid)

In [9]:
y_trainpredictions = []

for x in y_train_pred:
    y_trainpredictions.append(np.argmax(x))

y_validpredictions = []

for x in y_valid_pred:
    y_validpredictions.append(np.argmax(x))

In [10]:
from sklearn.metrics import accuracy_score, classification_report, precision_score, make_scorer

print('Train Accuracy: ', accuracy_score(y_train, y_trainpredictions))
print('Test Accuraccy: ', accuracy_score(y_valid, y_validpredictions))
print('Classification Report:')
print(classification_report(y_valid,y_validpredictions))

Train Accuracy:  0.9912962431889877
Test Accuraccy:  0.9270474879559532
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       826
           1       0.89      0.68      0.77       254
           2       0.97      0.65      0.78       310
           3       0.93      0.99      0.96     13102
           4       0.76      0.67      0.71       509
           5       0.80      0.75      0.78       668
           6       0.76      0.33      0.46       242
           7       0.97      0.93      0.95       729
           8       0.87      0.32      0.47       305
           9       0.97      0.56      0.71       214
          10       0.94      0.65      0.77       277

    accuracy                           0.93     17436
   macro avg       0.89      0.68      0.76     17436
weighted avg       0.92      0.93      0.92     17436



In [11]:
import joblib
from joblib import dump, load
# save model
joblib.dump(model_lgbm, 'gdrive/MyDrive/CS4220/Models/lgbm_model-final2.joblib') 

['gdrive/MyDrive/CS4220/Models/lgbm_model-final2.joblib']

hyperparameter

In [ ]:
from sklearn.model_selection import GridSearchCV
model = lightgbm.LGBMClassifier(objective='multiclass')
n_estimators=[500]
max_depth=[5]
lambda_l1=[0.0, 0.25, 1.0]
# lambda_l2=[0.25]
# subsample=[0.8]
bagging_freq=[20]
# colsample_bytree= [0.3]
num_leaves=[31]
boosting=['gbdt']
learning_rate=[0.1]
params = {
          'learning_rate':learning_rate,
          'n_estimators':n_estimators,
          'max_depth':max_depth, 
          'num_leaves':num_leaves,
          'boosting':boosting,
          # 'lambda_l1':lambda_l1,
          # 'lambda_l2':lambda_l2,
          'bagging_freq':bagging_freq}
optimal_params = GridSearchCV(
    estimator=model, param_grid=params, scoring='f1_macro', 
    verbose=3, n_jobs=None, cv=3 
)

optimal_params.fit(X_train, 
                   y_train, 
                   early_stopping_rounds=10,
                   eval_metric='logloss', 
                   eval_set=[(X_valid, y_valid)],
                   verbose = -1
                   
                   )

print(f'Best score: {optimal_params.best_score_}')
print(f'Best params: {optimal_params.best_params_}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Training until validation scores don't improve for 10 rounds.
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 0.248255	valid_0's multi_logloss: 0.248255
[CV 1/3] END bagging_freq=20, boosting=gbdt, lambda_l1=0.0, lambda_l2=0.25, learning_rate=0.1, max_depth=5, n_estimators=200, num_leaves=32;, score=0.727 total time= 5.6min
Training until validation scores don't improve for 10 rounds.
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 0.243306	valid_0's multi_logloss: 0.243306
[CV 2/3] END bagging_freq=20, boosting=gbdt, lambda_l1=0.0, lambda_l2=0.25, learning_rate=0.1, max_depth=5, n_estimators=200, num_leaves=32;, score=0.726 total time= 5.4min
Training until validation scores don't improve for 10 rounds.
Did not meet early stopping. Best iteration is:
[199]	valid_0's multi_logloss: 0.246968	valid_0's multi_logloss: 0.246968
[CV 3/3] END bagging_freq=20, boosting=gbdt

In [ ]:
tuned_parameters = {'objective': 'multiclass',
                    'num_class': 11,
                    'metric': 'multi_logloss',
                    'is_unbalance': 'true',
                    'boosting': 'gbdt',
                    'num_leaves': 32,
                    'max_depth':5,
                    # 'lambda_l1': 1.0,
                    # 'lambda_l2':0.25,
                    # 'feature_fraction': 0.5,
                    # 'bagging_fraction': 0.5,
                    'bagging_freq': 20,
                    'learning_rate': 0.1,
                    'n_estimators': 200,
                    'verbose': 1
                   }